In [111]:
from dotenv import load_dotenv

load_dotenv()

True

In [112]:
responses = [
  "Resilience is built through unity and the collective spirit of our community.",
  "I find strength by remembering the stories of our ancestors’ perseverance through tough times.",
  "Supporting each other in small everyday actions helps build a resilient community.",
  "By staying informed and engaged, I feel empowered to face challenges.",
  "Resilience comes from the hope and vision of a peaceful future for our nation.",
  "Educating our youth about our history and struggles helps foster resilience.",
  "I build resilience by maintaining cultural traditions that remind us of our identity and strength.",
  "Organizing community support activities and helping those in need gives me strength.",
  "Through art and creative expression, we can channel our emotions and rebuild our spirit.",
  "Personal resilience is built by facing fears directly and learning to overcome them.",
  "Sharing stories of survival and heroism inspires me to be resilient.",
  "Resilience means adapting to change while holding onto the values that define us.",
  "Physical wellness and mental health are key to maintaining resilience under stress.",
  "Building strong networks of communication and support are essential for resilience.",
  "I practice resilience by focusing on solutions rather than dwelling on problems.",
  "Volunteering in community defense initiatives has strengthened my resolve and resilience.",
  "Keeping a daily routine and setting small, achievable goals help maintain normalcy and resilience.",
  "Resilience is about recovering from setbacks and being ready to face the next challenge.",
  "Faith and spirituality play a crucial role in my personal resilience.",
  "Continuing to celebrate life’s moments, big and small, reinforces the resilience of our community.",
]

In [119]:
import spacy

# Load the pre-trained English model
nlp = spacy.load('en_core_web_sm')

# Function to perform NER
def extract_entities(sentences):
    for sentence in sentences:
        # Process the sentence using spaCy
        doc = nlp(sentence)

        # Function to find the actor, action, and target
        def extract_actor_action_target(doc):
            # Initialize empty placeholders
            actor, action, target = None, None, None
            
            # Iterate through the words in the sentence
            for token in doc:
                # Find the main verb (action)
                if token.dep_ == "ROOT" and token.pos_ == "VERB":
                    action = token.text
                    # Find the subject of the verb (actor)
                    for child in token.children:
                        if child.dep_ in ["nsubj", "nsubjpass"]:  # nsubjpass for passive subject
                            actor = child.text
                        # Find the object of the verb (target)
                        if child.dep_ in ["dobj", "attr", "prep", "pobj"]:
                            target = child.text
            return actor, action, target

        # Extract and print actor, action, target
        actor, action, target = extract_actor_action_target(doc)
        print(f"Action: {action}")

# Extract entities from the provided sentences
extract_entities(responses)


Actor: Resilience, 		 action: built, 		 target: through
Actor: I, 		 action: find, 		 target: by
Actor: None, 		 action: helps, 		 target: None
Actor: I, 		 action: feel, 		 target: By
Actor: Resilience, 		 action: comes, 		 target: from
Actor: None, 		 action: helps, 		 target: resilience
Actor: I, 		 action: build, 		 target: by
Actor: None, 		 action: gives, 		 target: strength
Actor: we, 		 action: channel, 		 target: emotions
Actor: resilience, 		 action: built, 		 target: by
Actor: None, 		 action: Sharing, 		 target: stories
Actor: Resilience, 		 action: means, 		 target: None
Actor: None, 		 action: None, 		 target: None
Actor: None, 		 action: None, 		 target: None
Actor: I, 		 action: practice, 		 target: by
Actor: Volunteering, 		 action: strengthened, 		 target: resolve
Actor: None, 		 action: help, 		 target: None
Actor: None, 		 action: None, 		 target: None
Actor: Faith, 		 action: play, 		 target: in
Actor: None, 		 action: reinforces, 		 target: resilience


In [114]:
import os
from neo4j import GraphDatabase, Driver, Transaction

# Create a driver instance
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)
# Optionally, check if you want to verify connectivity at the start
driver.verify_connectivity()

# Clear the database
driver.session().run("MATCH (n) DETACH DELETE n")

In [115]:
# Function to create nodes in the database
def create_response_nodes(driver:Driver, labels, responses):
    # Create a session using the driver
    with driver.session() as session:
        for i in set(labels):
            session.run("MERGE (:Cluster {id: $id, description: $description})", id=i, description=f"Cluster {i} Description")

        # Create responses and link to clusters
        for response, label in zip(responses, labels):
            session.run(
                """
                CREATE (r:Response {content: $content})
                WITH r
                MATCH (c:Cluster {id: $clusterId})
                CREATE (r)-[:BELONGS_TO]->(c)
                """,
                content=response, clusterId=label
            )

# Define a transaction function to execute a Cypher query
def create_node(tx:Transaction, response, label):
    cypher_query = """
    CREATE (n:Response {content: $content, cluster: $cluster})
    """
    tx.run(cypher_query, content=response, cluster=label)

# Call the function to create nodes
create_response_nodes(driver, labels, responses)

In [116]:
res = driver.execute_query("""
MATCH (n:Response)-[b:BELONGS_TO]->(c:Cluster)
RETURN n,b,c
""")

clusters = {}
for record in res.records:
    cluster = record['c']['id']
    content = record['n']['content']
    if cluster not in clusters:
        clusters[cluster] = []
    clusters[cluster].append(content)
    
clusters

{0: ['I find strength by remembering the stories of our ancestors’ perseverance through tough times.',
  'Sharing stories of survival and heroism inspires me to be resilient.',
  'Keeping a daily routine and setting small, achievable goals help maintain normalcy and resilience.'],
 1: ['Resilience is built through unity and the collective spirit of our community.',
  'Resilience comes from the hope and vision of a peaceful future for our nation.',
  'Organizing community support activities and helping those in need gives me strength.',
  'Through art and creative expression, we can channel our emotions and rebuild our spirit.',
  'Personal resilience is built by facing fears directly and learning to overcome them.',
  'Resilience means adapting to change while holding onto the values that define us.',
  'Building strong networks of communication and support are essential for resilience.',
  'I practice resilience by focusing on solutions rather than dwelling on problems.',
  'Volunteer

In [117]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
def extract_key_terms(data):
    vectorizer = CountVectorizer(stop_words='english')
    for cluster, texts in data.items():
        X = vectorizer.fit_transform(texts)
        terms = vectorizer.get_feature_names_out()
        frequencies = X.toarray().sum(axis=0)
        sorted_indices = np.argsort(frequencies)[::-1]
        print(f"Cluster {cluster} key terms: {terms[sorted_indices][:5]}")  # Top 5 terms

extract_key_terms(clusters)

Cluster 0 key terms: ['stories' 'tough' 'perseverance' 'ancestors' 'daily']
Cluster 1 key terms: ['resilience' 'community' 'spirit' 'support' 'built']
Cluster 2 key terms: ['face' 'staying' 'setbacks' 'resilience' 'recovering']
Cluster 3 key terms: ['helps' 'youth' 'supporting' 'struggles' 'small']
Cluster 4 key terms: ['resilience' 'maintaining' 'wellness' 'traditions' 'stress']
